# simulation


This simulation runs `numberOfSimulations` experiments and compares three different decision making strategies:

1. using statistical significance as a decision making criterion

2. using expected value as a decision making criterion

3. using a mixed strategy where statistical significance is a filter and expected value chooses the winner after filtering




In [1]:
val numberOfSimulations = Math.pow(10,4).toInt
// true population value
val baselineMean = 10
val baselineSD = 1
val standardDeviation = 5
val sampleSize = 999 / 3
val significanceThreshold = 0.05
val percentNullFromControl = 0.5
val percentNullFromOtherTreatment = (1 - percentNullFromControl) / 2


In [2]:
import org.apache.commons.math3.distribution.NormalDistribution
import org.apache.commons.math3.random.JDKRandomGenerator

val seed = 123
val rng = new JDKRandomGenerator(seed)

val baselineDistribution = new NormalDistribution(rng, baselineMean, baselineSD)

val effectSizeWhenNotNull = 0.005
val effectSizeFromOtherTreatment = 0.001


<br>

In [4]:
object Winner extends Enumeration {
  type Winner = Value
  val Control, Treatment1, Treatment2 = Value
}

case class ResultByStrategy(
  truth: Winner.Value, 
  nhst: Winner.Value, 
  mixed: Winner.Value, 
  expectedValue: Winner.Value
)

case class RealizedGain(trueObserved: Double, nhst: Double, mixed: Double, expectedValue: Double)

In [5]:
import org.apache.commons.math3.stat.inference.TTest

def determineNhstWinner(ctrlSamples: Array[Double], t1Samples: Array[Double], t2Samples: Array[Double], t1TrueMean: Double, t2TrueMean: Double): Winner.Value = {
    val tTest = new TTest()

    // Control vs Treatment 1
    val observedControlMean = ctrlSamples.sum/sampleSize
    val pValue1 = tTest.tTest(observedControlMean, t1Samples)

    // Control vs Treatment 2
    val pValue2 = tTest.tTest(observedControlMean, t2Samples)
    
    val observedT1Mean = t1Samples.sum/sampleSize
    val observedT2Mean = t2Samples.sum/sampleSize
    
    val t1SignificantlyBetter = (pValue1 < significanceThreshold && observedT1Mean > observedControlMean)
    val t2SignificantlyBetter = (pValue2 < significanceThreshold && observedT2Mean > observedControlMean)

    // runoff if needed
    if (t1SignificantlyBetter && t2SignificantlyBetter) {
        val t1Higher = observedT1Mean > observedT2Mean
        val controlMean = if (t1Higher) t2TrueMean else t1TrueMean
        val treatmentMean = if (t1Higher) t1TrueMean else t2TrueMean

        val ctrlSamples = new NormalDistribution(rng, controlMean, standardDeviation).sample(sampleSize)
        val treatSamples = new NormalDistribution(rng, treatmentMean, standardDeviation).sample(sampleSize)

        val pValue = tTest.tTest(ctrlSamples.sum/sampleSize, treatSamples)

        // if stat sig then treatment is winner otherwise stick with control (fail to reject null hypothesis)
        if (pValue < significanceThreshold) {
            if (t1Higher) Winner.Treatment1 else Winner.Treatment2
        } else {
            if (t1Higher) Winner.Treatment2 else Winner.Treatment1
        }
    } else if (t1SignificantlyBetter) {
        Winner.Treatment1
    } else if (t2SignificantlyBetter) {
        Winner.Treatment2
    } else {
        Winner.Control
    }
}

In [6]:
def determineMixedWinner(ctrlSamples: Array[Double], t1Samples: Array[Double], t2Samples: Array[Double]): Winner.Value = {
    val tTest = new TTest()

    val observedControlMean = ctrlSamples.sum/sampleSize
    val observedT1Mean = t1Samples.sum / sampleSize
    val observedT2Mean = t2Samples.sum / sampleSize

    // Control vs Treatment 1
    val pValue1 = tTest.tTest(observedControlMean, t1Samples)

    // Control vs Treatment 2
    val pValue2 = tTest.tTest(observedControlMean, t2Samples)

    val t1SignificantlyBetter = (pValue1 < significanceThreshold && observedT1Mean > observedControlMean)
    val t2SignificantlyBetter = (pValue2 < significanceThreshold && observedT2Mean > observedControlMean)

    // If they're both significant choose the higher ev
    if (t1SignificantlyBetter && t2SignificantlyBetter) {
        if (observedT1Mean > observedT2Mean) {
            Winner.Treatment1
        } else {
            Winner.Treatment2
        }
    } else if (t1SignificantlyBetter) {
        Winner.Treatment1
    } else if (t2SignificantlyBetter) {
        Winner.Treatment2
    } else {
        Winner.Control
    }
}

In [7]:
def determineEvWinner(ctrlSamples: Array[Double], t1Samples: Array[Double], t2Samples: Array[Double]): Winner.Value = {
    val observedCtrlMean = ctrlSamples.sum / sampleSize
    val observedT1Mean = t1Samples.sum / sampleSize
    val observedT2Mean = t2Samples.sum / sampleSize

    if (observedCtrlMean > observedT1Mean && observedCtrlMean > observedT2Mean) {
        Winner.Control
    } else if (observedT1Mean > observedT2Mean) {
        Winner.Treatment1
    } else {
        Winner.Treatment2
    }
}

In [8]:
val results: Seq[(ResultByStrategy, RealizedGain)] = for (i <- 0 until numberOfSimulations) yield {
    val tr1Mean = if (numberOfSimulations * percentNullFromControl < i) {
        baselineMean + (baselineMean * effectSizeWhenNotNull)
    } else {
        baselineMean
    }
    val tr1Baseline = new NormalDistribution(rng, tr1Mean, baselineSD)
    val tr2Mean = if (numberOfSimulations * (percentNullFromControl + percentNullFromOtherTreatment) < i) {
        baselineMean + (baselineMean * (effectSizeWhenNotNull + effectSizeFromOtherTreatment))
    } else {
        baselineMean
    }
    val tr2Baseline = new NormalDistribution(rng, tr2Mean, baselineSD)

    val ctrlDistribution = new NormalDistribution(rng, baselineDistribution.sample, standardDeviation)
    val t1Distribution = new NormalDistribution(rng, tr1Baseline.sample, standardDeviation)
    val t2Distribution = new NormalDistribution(rng, tr2Baseline.sample, standardDeviation)

    val ctrlSamples = ctrlDistribution.sample(sampleSize)
    val t1Samples = t1Distribution.sample(sampleSize)
    val t2Samples = t2Distribution.sample(sampleSize)

    val nhstWinner = determineNhstWinner(ctrlSamples, t1Samples, t2Samples, t1Distribution.getMean, t2Distribution.getMean)
    val mixedWinner = determineMixedWinner(ctrlSamples, t1Samples, t2Samples)
    val evWinner = determineEvWinner(ctrlSamples, t1Samples, t2Samples)

    //ctrlSamples.take(3).foreach(System.out.println)
    val trueBestSample = if (ctrlDistribution.getMean >= t1Distribution.getMean && ctrlDistribution.getMean >= t2Distribution.getMean) {
        Winner.Control
    } else if (t1Distribution.getMean > t2Distribution.getMean) {
        Winner.Treatment1
    } else {
        Winner.Treatment2
    }

    val trueBest = if (baselineMean >= tr1Mean && baselineMean >= tr2Mean) {
        Winner.Control
    } else if (tr1Mean >= tr2Mean) {
        Winner.Treatment1
    } else {
        Winner.Treatment2
    }

    //if (i > 9000) {
    //    println(i.toString + "?" + (numberOfSimulations * percentNullFromControl > i) + "," + (numberOfSimulations * percentNullFromControl).toString + " - " + (numberOfSimulations * (percentNullFromControl + percentNullFromOtherTreatment)).toString + " : " + tr1Multiplier.toString + " - " + tr2Multiplier.toString)
    //}

    def winGain(strategyWinner: Winner.Value): Double = {
        if (strategyWinner == Winner.Treatment1) {
            tr1Mean - baselineMean
        } else if (strategyWinner == Winner.Treatment2) {
            tr2Mean - baselineMean
        } else {
            0
        }
    }
    
    if (nhstWinner != mixedWinner) {
        println((ctrlSamples.sum / sampleSize).toString + " - " + (t1Samples.sum / sampleSize).toString + " - " + (t2Samples.sum / sampleSize).toString)
        println(nhstWinner.toString + " - " + mixedWinner.toString + " - " + evWinner.toString)
    }

    (
        ResultByStrategy(trueBest, nhstWinner, mixedWinner, evWinner),
        RealizedGain(
            winGain(trueBest),
            winGain(nhstWinner),
            winGain(mixedWinner),
            winGain(evWinner)
        )
    )
}

8.63897161073099 - 10.75743729951471 - 10.331362182315575
Treatment2 - Treatment1 - Treatment1
9.461123312526892 - 10.339051238082964 - 10.408591281035514
Treatment1 - Treatment2 - Treatment2
8.884643469989763 - 9.810612881113721 - 9.646504125386855
Treatment2 - Treatment1 - Treatment1
9.707748174273299 - 10.925838963520075 - 11.007953292968566
Treatment1 - Treatment2 - Treatment2
8.755676461116042 - 10.73444395196693 - 10.835927092212996
Treatment1 - Treatment2 - Treatment2
9.255882832993427 - 10.689653517756858 - 10.03891313138308
Treatment2 - Treatment1 - Treatment1
9.183779099102237 - 10.500822609912177 - 10.10210614113521
Treatment2 - Treatment1 - Treatment1
8.81881718054947 - 9.57822180878473 - 10.303321272198552
Treatment1 - Treatment2 - Treatment2
8.399064387186556 - 9.584633145530809 - 9.502477418799964
Treatment2 - Treatment1 - Treatment1
9.055915903654755 - 10.66492254625022 - 10.667565481485521
Treatment1 - Treatment2 - Treatment2
7.875906976944931 - 10.499560367840239 - 10

In [9]:
// Debug Step
results.take(100).map(_._1).map(_.nhst).foreach(println)
results.map(_._1).count(_.nhst != Winner.Control)

Treatment1
Control
Control
Control
Treatment1
Treatment2
Control
Control
Control
Treatment1
Treatment2
Treatment1
Control
Treatment1
Control
Treatment2
Treatment2
Control
Treatment1
Treatment1
Control
Control
Treatment1
Control
Control
Treatment1
Treatment2
Control
Treatment2
Treatment2
Control
Control
Treatment2
Treatment2
Control
Treatment1
Treatment2
Control
Control
Control
Treatment1
Control
Control
Control
Control
Control
Control
Treatment1
Control
Control
Treatment1
Treatment2
Treatment1
Treatment2
Treatment1
Control
Treatment1
Treatment1
Treatment2
Treatment2
Treatment1
Treatment1
Treatment1
Treatment1
Control
Treatment1
Treatment1
Treatment1
Control
Control
Treatment2
Treatment2
Control
Treatment2
Treatment2
Treatment1
Control
Treatment2
Treatment1
Treatment2
Control
Treatment1
Treatment2
Control
Treatment1
Treatment1
Treatment1
Control
Treatment1
Control
Control
Control
Treatment2
Treatment2
Treatment2
Control
Treatment1
Treatment2
Control
Treatment1


5165

In [10]:
val agreesWithTruth = results.map(_._1).map{ res =>
    (res.truth == res.nhst,
    res.truth == res.mixed,
    res.truth == res.expectedValue)
}

println(agreesWithTruth.count(_._1))
println(agreesWithTruth.count(_._2))
println(agreesWithTruth.count(_._3))

3784
3810
3466


In [11]:
val agreesWithStatSig: Seq[(Boolean, Boolean)] = results.map(_._1).map{ res =>
    (res.nhst == res.mixed,
    res.nhst == res.expectedValue)
}

def toInt(b: Boolean) = if (b) 1 else 0

println(agreesWithStatSig.count(_._1))
println(agreesWithStatSig.count(_._2))

val chartDataStatSig = agreesWithStatSig.foldLeft(List(("Mixed",0),("Expected Value",0)))((acc, elem) => {
    List((acc(0)._1, acc(0)._2 + toInt(elem._1)), (acc(1)._1, acc(1)._2 + toInt(elem._2)))
})
chartDataStatSig

9260
7775


List((Mixed,9260), (Expected Value,7775))

In [12]:
val agreesWithMixed = results.map(_._1).map{ res =>
    (res.mixed == res.nhst,
    res.mixed == res.expectedValue)
}

println(agreesWithMixed.count(_._1))
println(agreesWithMixed.count(_._2))

val chartDataMixed = agreesWithMixed.foldLeft(List(("Stat Sig",0),("Expected Value",0)))((acc, elem) => {
    List((acc(0)._1, acc(0)._2 + toInt(elem._1)), (acc(1)._1, acc(1)._2 + toInt(elem._2)))
})
chartDataMixed

9260
8515


List((Stat Sig,9260), (Expected Value,8515))

In [13]:
val agreesWithExpectedValue = results.map(_._1).map{ res =>
    (res.expectedValue == res.nhst,
    res.expectedValue == res.mixed)
}

println(agreesWithExpectedValue.count(_._1))
println(agreesWithExpectedValue.count(_._2))

val chartDataEV = agreesWithMixed.foldLeft(List(("Stat Sig",0),("Mixed",0)))((acc, elem) => {
    List((acc(0)._1, acc(0)._2 + toInt(elem._1)), (acc(1)._1, acc(1)._2 + toInt(elem._2)))
})
chartDataEV

7775
8515


List((Stat Sig,9260), (Mixed,8515))

In [14]:
val whichChart = chartDataMixed
val xAxisLabel = "Agreeing with Mixed Strategy"

In [15]:
{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "description": "A bar chart with three bars and a reference line",
  "width": 300,
  "height": 600,
    "data": {
        "name": "chartData",
        "values": whichChart
    },
    "layer": [
    {
      "mark": {
        "type": "bar",
        "cornerRadius": 8,
        "tooltip": true
      },
      "encoding": {
        "x": {
          "field": "_1",
          "type": "nominal",
          "axis": {
            "title": "Decision Strategy",
            "labelAngle": 0,
            "labelPadding": 10,
            "tickSize": 0,
            "domainWidth": 0
          }
        },
        "y": {
          "field": "_2",
          "type": "quantitative",
          "axis": {
            "title": xAxisLabel,
            "grid": false,
            "tickCount": 5
          }
        },
        "color": {
          "field": "_1",
          "type": "nominal",
          "scale": {
            "range": ["#FFB3BA", "#BAFFC9", "#BAE1FF"]
          },
          "legend": null
        }
      }
    },
    {
      "mark": {
        "type": "rule",
        "strokeDash": [6, 4],
        "stroke": "#666666",
        "strokeWidth": 1
      },
      "encoding": {
        "y": {"datum": 10000}
      }
    }
  ],
  "config": {
    "view": {
      "stroke": null
    },
    "axis": {
      "labelFont": "Arial",
      "labelFontSize": 12,
      "titleFont": "Arial",
      "titleFontSize": 14,
      "titlePadding": 20
    },
    "background": "#FFFFFF"
  }
}

In [16]:
results.take(100).map(_._2).foreach(println)
println(results.map(_._2).map(_.trueObserved).sum)
println(results.map(_._2).map(_.nhst).sum)
println(results.map(_._2).map(_.mixed).sum)
println(results.map(_._2).map(_.expectedValue).sum)

RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGain(0.0,0.0,0.0,0.0)
RealizedGa